# Wrangle the data:

### Vector and attribute:
**Socioeconomic data** in .csv format has been retreived from the City of Toronto's Open Data Portal. However, this data is not appropriately formatted for our intents and purposes. Additionally, it lacks a geometry field. As such, it will be *cleaned*, *filtered*, *reshaped*, *merged* with spatial data from a GeoJSON neighbourhood boundaries file, and finally *transformed*.

By this point we will have fully prepped our median income by neighbourhood data set and will move onto derving the land surface temperature.

### LST

In [175]:
# Install libraries
!pip install Pandas
!pip install GeoPandas

In [176]:
# Add Libraries
import pandas as pd
import geopandas as gpd
import re

In [177]:
# Read the socioeconomic data into a DataFrame and preview the first 10 rows:

neighbourhood_profiles_df = pd.read_csv("Data/to_nbh_profiles_2021.csv", header = 0)
neighbourhood_profiles_df.head(10)

,Neighbourhood Name,West Humber-Clairville,Mount Olive-Silverstone-Jamestown,Thistletown-Beaumond Heights,Rexdale-Kipling,Elms-Old Rexdale,Kingsview Village-The Westway,Willowridge-Martingrove-Richview,Humber Heights-Westmount,Edenbridge-Humber Valley,...,Harbourfront-CityPlace,St Lawrence-East Bayfront-The Islands,Church-Wellesley,Downtown Yonge East,Bay-Cloverhill,Yonge-Bay Corridor,Junction-Wallace Emerson,Dovercourt Village,North Toronto,South Eglinton-Davisville
0,Neighbourhood Number,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,174
1,TSNS 2020 Designation,Not an NIA or Emerging Neighbourhood,Neighbourhood Improvement Area,Neighbourhood Improvement Area,Not an NIA or Emerging Neighbourhood,Neighbourhood Improvement Area,Neighbourhood Improvement Area,Not an NIA or Emerging Neighbourhood,Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,...,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood
2,Total - Age groups of the population - 25% sam...,33300,31345,9850,10375,9355,22005,22445,10005,15190,...,28135,31285,22320,17700,16670,12645,23180,12380,15885,22735
3,0 to 14 years,4295,5690,1495,1575,1610,3915,3500,1370,2070,...,2065,2285,895,1055,745,970,3075,1365,1315,2190
4,0 to 4 years,1460,1650,505,505,440,1245,1065,395,520,...,1030,1045,495,480,370,500,1135,445,535,910
5,5 to 9 years,1345,1860,540,615,480,1325,1190,430,740,...,655,690,230,325,255,270,1020,430,390,670
6,10 to 14 years,1485,2175,455,455,685,1350,1240,540,815,...,385,550,175,245,120,200,925,490,390,610
7,15 to 64 years,23640,21490,6615,6950,6355,14385,13865,6245,9650,...,24540,25015,19315,14695,13815,10820,17200,9040,12780,17495
8,15 to 19 years,1860,2280,570,515,635,1245,1175,525,885,...,365,600,465,390,535,340,750,460,465,575
9,20 to 24 years,3175,2675,745,715,685,1605,1330,580,780,...,2250,1990,3010,2340,3810,1825,1185,640,960,1115


In [178]:
# Keep only the rows for "Neighbourhood Name", "Neighbourhood Number", and "Median total income in 2020 among recipients ($)":

#Initailize a list of names
row_names = [
    "Median total income in 2020  among recipients ($)",
    "Neighbourhood Name",
    "Neighbourhood Number"
]

#Format them for regex
pattern = "|".join([re.escape(name) for name in row_names])

filtered_df = neighbourhood_profiles_df[
    neighbourhood_profiles_df['Neighbourhood Name'].str.contains(pattern, na=False, regex=True)
]

#Reset the index
filtered_df = filtered_df.reset_index(drop = True)

filtered_df.head()

,Neighbourhood Name,West Humber-Clairville,Mount Olive-Silverstone-Jamestown,Thistletown-Beaumond Heights,Rexdale-Kipling,Elms-Old Rexdale,Kingsview Village-The Westway,Willowridge-Martingrove-Richview,Humber Heights-Westmount,Edenbridge-Humber Valley,...,Harbourfront-CityPlace,St Lawrence-East Bayfront-The Islands,Church-Wellesley,Downtown Yonge East,Bay-Cloverhill,Yonge-Bay Corridor,Junction-Wallace Emerson,Dovercourt Village,North Toronto,South Eglinton-Davisville
0,Neighbourhood Number,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,174
1,Median total income in 2020 among recipie...,33600,29600,32800,33600,34400,34400,40800,40400,48400,...,58800,58000,42800,45200,39600,44000,41200,38000,46000,52400


In [179]:
# Transpose the filtered df:

transposed_df = filtered_df.T

transposed_df.head()

,0,1
Neighbourhood Name,Neighbourhood Number,Median total income in 2020 among recipie...
West Humber-Clairville,1,33600
Mount Olive-Silverstone-Jamestown,2,29600
Thistletown-Beaumond Heights,3,32800
Rexdale-Kipling,4,33600


In [180]:
# Reset the indices and headers:

transposed_df = transposed_df.reset_index()

transposed_df.columns = transposed_df.iloc[0]
result_df = transposed_df[1:].reset_index(drop = True)

result_df.head()

,Neighbourhood Name,Neighbourhood Number,Median total income in 2020 among recipients ($)
0,West Humber-Clairville,1,33600
1,Mount Olive-Silverstone-Jamestown,2,29600
2,Thistletown-Beaumond Heights,3,32800
3,Rexdale-Kipling,4,33600
4,Elms-Old Rexdale,5,34400


In [181]:
# Format the transposed df:

# Rename
result_df.columns = ["nbh_name", "nbh_num", "med_inc"]


# Ensure no whitespace
result_df.columns = result_df.columns.str.strip()

# Drop neighbourhood number as it is irrelivent in the merge
result_df = result_df.drop("nbh_num", axis = 1)

result_df


,nbh_name,med_inc
0,West Humber-Clairville,33600
1,Mount Olive-Silverstone-Jamestown,29600
2,Thistletown-Beaumond Heights,32800
3,Rexdale-Kipling,33600
4,Elms-Old Rexdale,34400
...,...,...
153,Yonge-Bay Corridor,44000
154,Junction-Wallace Emerson,41200
155,Dovercourt Village,38000
156,North Toronto,46000


In [182]:
# Normalize neighbourhood name formatting for merge:

result_df["nbh_name"] = result_df["nbh_name"].str.replace(' ', '').str.replace('-', '').str.replace("'","").str.replace(".","").str.replace("`","").str.strip().str.lower()

result_df

,nbh_name,med_inc
0,westhumberclairville,33600
1,mountolivesilverstonejamestown,29600
2,thistletownbeaumondheights,32800
3,rexdalekipling,33600
4,elmsoldrexdale,34400
...,...,...
153,yongebaycorridor,44000
154,junctionwallaceemerson,41200
155,dovercourtvillage,38000
156,northtoronto,46000


In [183]:
# Export to .csv

result_df.to_csv("Data/attribute_data.csv", index = False)

**Now we have successfully cleaned the data and outputted the result to a .csv.**

**Key Learnings:**
    -Watch out for spaces and special characters when parsing data --> regex is an effective solution!
    -Be catious with headers and indices when transposing tabular data.

**The next objective is to merge this data with the corresponding spatial data in the "neighbourhood_boundariesWGS84.geojson" file.**

In [184]:
# Opne the GeoJSON containing the neighbourhood boundaries:

geo_df = gpd.read_file("Data/neighbourhood_boundariesWGS84.geojson")

geo_df.to_csv("Data/nbh_bnds_tst.csv")

In [185]:
# Read the attribute data into a DataFrame:

attribute_df = pd.read_csv("Data/attribute_data.csv")

attribute_df.head()

,nbh_name,med_inc
0,westhumberclairville,33600
1,mountolivesilverstonejamestown,29600
2,thistletownbeaumondheights,32800
3,rexdalekipling,33600
4,elmsoldrexdale,34400


In [186]:
# Normalize the GeoJSON neighbourhood name formatting for merge:

geo_df["AREA_NAME"] = geo_df["AREA_NAME"].str.replace(' ', '').str.replace('-', '').str.replace("'","").str.replace(".","").str.replace("`","").str.strip().str.lower()

geo_df

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,CLASSIFICATION,CLASSIFICATION_CODE,OBJECTID,geometry
0,1,2502366,26022881,0,174,174,southeglintondavisville,South Eglinton-Davisville (174),Not an NIA or Emerging Neighbourhood,NA,17824737.0,"MULTIPOLYGON (((-79.38635 43.69783, -79.38623 ..."
1,2,2502365,26022880,0,173,173,northtoronto,North Toronto (173),Not an NIA or Emerging Neighbourhood,NA,17824753.0,"MULTIPOLYGON (((-79.39744 43.70693, -79.39837 ..."
2,3,2502364,26022879,0,172,172,dovercourtvillage,Dovercourt Village (172),Not an NIA or Emerging Neighbourhood,NA,17824769.0,"MULTIPOLYGON (((-79.43411 43.66015, -79.43537 ..."
3,4,2502363,26022878,0,171,171,junctionwallaceemerson,Junction-Wallace Emerson (171),Not an NIA or Emerging Neighbourhood,NA,17824785.0,"MULTIPOLYGON (((-79.4387 43.66766, -79.43841 4..."
4,5,2502362,26022877,0,170,170,yongebaycorridor,Yonge-Bay Corridor (170),Not an NIA or Emerging Neighbourhood,NA,17824801.0,"MULTIPOLYGON (((-79.38404 43.64497, -79.38502 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
153,154,2502213,26022728,0,001,001,westhumberclairville,West Humber-Clairville (1),Not an NIA or Emerging Neighbourhood,NA,17827185.0,"MULTIPOLYGON (((-79.59037 43.73401, -79.58942 ..."
154,155,2502212,26022727,0,024,024,blackcreek,Black Creek (24),Neighbourhood Improvement Area,NIA,17827201.0,"MULTIPOLYGON (((-79.51915 43.77399, -79.51901 ..."
155,156,2502211,26022726,0,023,023,pelmoparkhumberlea,Pelmo Park-Humberlea (23),Not an NIA or Emerging Neighbourhood,NA,17827217.0,"MULTIPOLYGON (((-79.53225 43.73505, -79.52938 ..."
156,157,2502210,26022725,0,022,022,humbermede,Humbermede (22),Neighbourhood Improvement Area,NIA,17827233.0,"MULTIPOLYGON (((-79.52813 43.74425, -79.52721 ..."


In [187]:
# Merge the two datasets:

merged_df = geo_df.merge(attribute_df, left_on = "AREA_NAME", right_on = "nbh_name")

merged_df

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,CLASSIFICATION,CLASSIFICATION_CODE,OBJECTID,geometry,nbh_name,med_inc
0,1,2502366,26022881,0,174,174,southeglintondavisville,South Eglinton-Davisville (174),Not an NIA or Emerging Neighbourhood,NA,17824737.0,"MULTIPOLYGON (((-79.38635 43.69783, -79.38623 ...",southeglintondavisville,52400
1,2,2502365,26022880,0,173,173,northtoronto,North Toronto (173),Not an NIA or Emerging Neighbourhood,NA,17824753.0,"MULTIPOLYGON (((-79.39744 43.70693, -79.39837 ...",northtoronto,46000
2,3,2502364,26022879,0,172,172,dovercourtvillage,Dovercourt Village (172),Not an NIA or Emerging Neighbourhood,NA,17824769.0,"MULTIPOLYGON (((-79.43411 43.66015, -79.43537 ...",dovercourtvillage,38000
3,4,2502363,26022878,0,171,171,junctionwallaceemerson,Junction-Wallace Emerson (171),Not an NIA or Emerging Neighbourhood,NA,17824785.0,"MULTIPOLYGON (((-79.4387 43.66766, -79.43841 4...",junctionwallaceemerson,41200
4,5,2502362,26022877,0,170,170,yongebaycorridor,Yonge-Bay Corridor (170),Not an NIA or Emerging Neighbourhood,NA,17824801.0,"MULTIPOLYGON (((-79.38404 43.64497, -79.38502 ...",yongebaycorridor,44000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,154,2502213,26022728,0,001,001,westhumberclairville,West Humber-Clairville (1),Not an NIA or Emerging Neighbourhood,NA,17827185.0,"MULTIPOLYGON (((-79.59037 43.73401, -79.58942 ...",westhumberclairville,33600
154,155,2502212,26022727,0,024,024,blackcreek,Black Creek (24),Neighbourhood Improvement Area,NIA,17827201.0,"MULTIPOLYGON (((-79.51915 43.77399, -79.51901 ...",blackcreek,29400
155,156,2502211,26022726,0,023,023,pelmoparkhumberlea,Pelmo Park-Humberlea (23),Not an NIA or Emerging Neighbourhood,NA,17827217.0,"MULTIPOLYGON (((-79.53225 43.73505, -79.52938 ...",pelmoparkhumberlea,39600
156,157,2502210,26022725,0,022,022,humbermede,Humbermede (22),Neighbourhood Improvement Area,NIA,17827233.0,"MULTIPOLYGON (((-79.52813 43.74425, -79.52721 ...",humbermede,32400


In [188]:
# Filter for the _id, neighbourhood_name, 2020_median_income, and geometry columns:

# Filter 
filtered_df = merged_df[["_id", "nbh_name", "med_inc", "geometry"]]

# Rename unique_id to id
result_df = filtered_df.rename(columns = {"_id": "id"})

result_df

,id,nbh_name,med_inc,geometry
0,1,southeglintondavisville,52400,"MULTIPOLYGON (((-79.38635 43.69783, -79.38623 ..."
1,2,northtoronto,46000,"MULTIPOLYGON (((-79.39744 43.70693, -79.39837 ..."
2,3,dovercourtvillage,38000,"MULTIPOLYGON (((-79.43411 43.66015, -79.43537 ..."
3,4,junctionwallaceemerson,41200,"MULTIPOLYGON (((-79.4387 43.66766, -79.43841 4..."
4,5,yongebaycorridor,44000,"MULTIPOLYGON (((-79.38404 43.64497, -79.38502 ..."
...,...,...,...,...
153,154,westhumberclairville,33600,"MULTIPOLYGON (((-79.59037 43.73401, -79.58942 ..."
154,155,blackcreek,29400,"MULTIPOLYGON (((-79.51915 43.77399, -79.51901 ..."
155,156,pelmoparkhumberlea,39600,"MULTIPOLYGON (((-79.53225 43.73505, -79.52938 ..."
156,157,humbermede,32400,"MULTIPOLYGON (((-79.52813 43.74425, -79.52721 ..."


In [189]:
# Export to a shapefile for visualiztion in GIS (QGIS in this case)

result_df.to_file("Data/socioeconomic/toronto_med_inc_2020.shp")